# QC Tutorial

---

This interactive tutorial will show you a couple ways to quality control data using LROSE using RadxPid and RadxQc. RadxPid allows users to censor non-weather echoes using the PID algorithm and RadxQc 

RadxPid is one of three 'Echo' applications in LROSE. A visual comparison of RadxPid with RadxKdp and RadxRate and their parameter files is shown below. Each application has its own main parameter file that defines variable names and specifies the paths to the parameter file for each relevant sub-process. For example, the main RadxPid parameter file links to the KDP and PID parameter files (the PID parameter file links to the PID thresholds file).

<div>
<img src="../images/radx_echo_description.png" width="600"/>
</div>

---

*Note: this tutorial shows two examples of how to quality control radar data. Other workflows exist!*

---

## Tutorial Overview
### 1. Setup

#### Directory organization

The structure of the echo tutorial on JupyterHub is shown in the diagram below. The parent or base directory is "ams2023" and contains all of the notebooks, parameter files, and data for the workshop.

<div>
<img src="../images/RadxRate_structure.png" width="500"/>
</div>

#### Download raw data and parameter files

CfRadial data file that will be downloaded:
* cfrad.20210815_032510.928_to_20210815_032947.216_CSU-CHILL_SUR.nc

Parameter files (included in this tutorial):

*Note: The parameter files have already been modified to run straight out of the box.* 
* RadxPid main params
* RadxPid Kdp_specific params
* RadxPid Pid_specific params
* Pid Thresholds params (S-band, simultaneous transmit)
* RadxQc params

### 2. Run RadxPid - censor non-weather gates

* Run PID algorithm to identify gates which do not have weather
    * RadxPid

### 3. Run RadxQc - censor based on thresholds

* Censor gates where RHOHV and NCP don't meet thresholds
    * RadxQc

### 4. Plot PID and Rate

* Visualize results of RadxPid and RadxQc analysis using Py-ART

---

For this tutorial, the parameter files have already been created and populated with the appropriate parameters. However, the default parameter files can be saved using the following commands.

In [ ]:
# # create original parameter files
# !/usr/local/lrose/bin/RadxPid -print_params > /home/jovyan/ams2023/params/user/RadxPid_main_params_user
# !/usr/local/lrose/bin/RadxPid -print_params_kdp > /home/jovyan/ams2023/params/user/RadxPid_Kdp_params_user
# !/usr/local/lrose/bin/RadxPid -print_params_pid > /home/jovyan/ams2023/params/user/RadxPid_Pid_params_user
# !/usr/local/lrose/bin/RadxQc -print_params > /home/jovyan/ams2023/params/user/RadxQc_params_user


# 1. Setup
## Environment and packages

First, we import the required python packages to run this notebook. Most of the LROSE processing can be done with the os package and shell commands. At the end we will plot the output using Py-ART.

In [ ]:
import os
import pyart
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import numpy as np

Next, we set up the directory structure to simplify our commands. If you are running this notebook at the LROSE workshop on JupyterHub, these paths go to the parent directory containing all the workshop resources and the LROSE binaries. 

**If you have downloaded this notebook, please modify BASE_DIR and LROSE_DIR to work on your personal machine.**

* BASE_DIR: the base directory containing the directories for the notebooks, data, parameter files
* RADAR_NAME: the name of the radar used in this tutorial
* LROSE_DIR: path to the LROSE installation 

In [ ]:
os.environ['BASE_DIR'] = '/home/jovyan/ams2023'
os.environ['RADAR_NAME'] = 'CHILL_S'
os.environ['LROSE_DIR'] = '/usr/local/lrose/bin'
base_dir = os.environ['BASE_DIR']
radar_name = os.environ['RADAR_NAME']
!echo "Base directory: "$BASE_DIR
!echo "Radar name: "$RADAR_NAME
!echo "LROSE directory: "$LROSE_DIR

## Data download and directory set up

We need to set up the required data directories and download the radar data and GFS analysis to the JupyterHub. We delete any existing files and directories specific to this tutorial to ensure we're starting with clean directories and files.

In [ ]:
## make subdirectory within data for the raw data
!rm -rf ${BASE_DIR}/data/qc/raw
!mkdir -p ${BASE_DIR}/data/qc/raw

## wget CfRadial files from EOL server
!wget http://front.eol.ucar.edu/data/notebooks/ams2023/cfrad.20210815_032510.928_to_20210815_032947.216_CSU-CHILL_SUR.nc

## move files to proper directory
!mv cfrad.20210815_032510.928_to_20210815_032947.216_CSU-CHILL_SUR.nc ${BASE_DIR}/data/qc/raw


# 2.  Run RadxPid - censor non-weather gates

## RadxPid params

- **RadxPid main params**
    - insert text

In [ ]:
!rm -rf ${BASE_DIR}/data/qc/radxpid
!mkdir -p ${BASE_DIR}/data/qc/radxpid

# run RadxPid
#!${LROSE_DIR}/RadxPid -vv -params ${BASE_DIR}/params/qc/RadxPid_main_params -f ${BASE_DIR}/data/qc/raw/*.nc -outdir ${BASE_DIR}/data/qc/radxpid
!${LROSE_DIR}/RadxRate -params ${BASE_DIR}/params/qc/RadxRate_main_params -f ${BASE_DIR}/data/qc/raw/*.nc -outdir ${BASE_DIR}/data/qc/radxpid



# 3. Run RadxQc - censor based on thresholds

****

Here, *-params* provides the link to the main RadxQc parameter file, *-f* provides the link to the files we want to process, and *-outdir* indicates where RadxRate should write the final files.

In [ ]:
# run RadxQc
!${LROSE_DIR}/RadxQc -params ${BASE_DIR}/params/qc/RadxQc_params -f ${BASE_DIR}/data/qc/raw/*.nc -outdir ${BASE_DIR}/data/qc/radxqc


# 4. Plot censored CHILL output

## Censoring using the PID

To visualize the output in the notebook, we can use Py-ART. HawkEye is also great for visualizing data - a general parameter file can be found in the echo parameter files directory.

In [ ]:
# Read CfRadial file into radar object
inDir = base_dir+"/data/qc/radxpid/20210815/"
file = "cfrad.20210815_032510.928_to_20210815_032947.216_CSU-CHILL_SUR.nc"
rate_kmhx = pyart.io.read_cfradial(inDir+file)
rate_kmhx.info('compact')


We can create a colormap for visualizing PID.

In [ ]:
pidmap = np.array([[0.12156862745098039, 0.46666666666666667, 0.70588235294117652, 1.0],
              [0.68235294117647061, 0.7803921568627451, 0.90980392156862744, 1.0],
              [0.59607843137254901, 0.87450980392156863, 0.54117647058823526, 1.0],
              [0.45490196078431372, 0.7686274509803922, 0.46274509803921571, 1.0],
              [0.17254901960784313, 0.62745098039215685, 0.17254901960784313, 1.0],
              [0.83921568627450982, 0.15294117647058825, 0.15686274509803921, 1.0],
              [1.0, 0.59607843137254901, 0.58823529411764708, 1.0],
              [1.0, 0.49803921568627452, 0.054901960784313725, 1.0],
              [1.0, 0.73333333333333328, 0.47058823529411764, 1.0],
              [0.61960784313725492, 0.85490196078431369, 0.89803921568627454, 1.0],
              [0.090196078431372548, 0.74509803921568629, 0.81176470588235294, 1.0],
              [0.61176470588235299, 0.61960784313725492, 0.87058823529411766, 1.0],
              [0.32156862745098042, 0.32941176470588235, 0.63921568627450975, 1.0],
              [0.859375, 0.859375, 0.859375, 1.0],
              [0.66015625, 0.66015625, 0.66015625, 1.0],
              [0.41015625, 0.41015625, 0.41015625, 1.0],
              [0.0, 0.0, 0.0, 1.0],],'f')
my_cmap2 = colors.ListedColormap(pidmap, name='ncar_pid')


In [ ]:
# Plot results of RadxRate

displayRate = pyart.graph.RadarDisplay(rate_kmhx)
figRate = plt.figure(1, (12, 10))

# DBZ (input)

axDbz = figRate.add_subplot(221)
displayRate.plot_ppi('DBZ', 0, vmin=-32, vmax=64.,
                    axislabels=("x(km)", "y(km)"),
                    colorbar_label="DBZ")
displayRate.plot_range_rings([50, 100, 150, 200, 250])
displayRate.plot_cross_hair(150.)
displayRate.set_limits(xlim=(-150,150),ylim=(-150,150))

# KDP (computed)

axKdp = figRate.add_subplot(222)
displayRate.plot_ppi('KDP', 0, vmin=0, vmax=2.,
    axislabels=("x(km)", "y(km)"),
    colorbar_label="KDP (deg/km)",
    cmap="nipy_spectral")
displayRate.plot_range_rings([50, 100, 150, 200, 250])
displayRate.plot_cross_hair(150.)
displayRate.set_limits(xlim=(-150,150),ylim=(-150,150))

# RATE_HYBRID (computed)

axHybrid = figRate.add_subplot(223)
displayRate.plot_ppi('ZDR', 0, vmin=-1., vmax=5.,
    axislabels=("x(km)", "y(km)"),
    colorbar_label="ZDR",
    cmap="nipy_spectral")
displayRate.plot_range_rings([50, 100, 150, 200, 250])
displayRate.plot_cross_hair(150.)
displayRate.set_limits(xlim=(-150,150),ylim=(-150,150))

# NCAR PID (computed)

axPID = figRate.add_subplot(224)
displayRate.plot_ppi('PID', 0, vmin=0.5, vmax = 17.5,
    axislabels=("x(km)", "y(km)"),
    colorbar_label="PID",
    cmap = my_cmap2, mask_outside=True)
displayRate.plot_range_rings([50, 100, 150, 200, 250])
displayRate.plot_cross_hair(150.)
displayRate.set_limits(xlim=(-150,150),ylim=(-150,150))

# plot all 17 PID categories
pid_cbar = displayRate.cbs[3]
#pid_cbar.set_ticks([1,2,3,4,5,6,7,8,9,10,11,12,13])
#pid_cbar.set_ticklabels(['cld-drops', 'drizzle', 'lt-rain', 'mod-rain', 'hvy-rain', 'hail', 'rain/hail', 'sm-hail', 'gr/rain', 'dry-snow', 'wet-snow', 'ice', 'irreg-ice'])
pid_cbar.set_ticks([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17])
pid_cbar.set_ticklabels(['cld-drops', 'drizzle', 'lt-rain', 'mod-rain', 'hvy-rain', 'hail', 'rain/hail', 'sm-hail', 'gr/rain', 'dry-snow', 'wet-snow', 'ice', 'irreg-ice', 'slw', 'insects', '2nd-trip', 'clutter'])

figRate.tight_layout()

plt.show()


## Censoring using the variable thresholds

In [ ]:
# Read CfRadial file into radar object
inDir = base_dir+"/data/qc/radxqc/20210815/"
file = "cfrad.20210815_032510.928_to_20210815_032947.216_CSU-CHILL_SUR.nc"
qc_chill = pyart.io.read_cfradial(inDir+file)
qc_chill.info('compact')

In [ ]:
# Plot results of RadxRate

displayRate = pyart.graph.RadarDisplay(qc_chill)
figRate = plt.figure(1, (12, 10))

# DBZ (input)

axDbz = figRate.add_subplot(221)
displayRate.plot_ppi('DBZ', 0, vmin=-32, vmax=64.,
                    axislabels=("x(km)", "y(km)"),
                    colorbar_label="DBZ")
displayRate.plot_range_rings([50, 100, 150, 200, 250])
displayRate.plot_cross_hair(150.)
displayRate.set_limits(xlim=(-150,150),ylim=(-150,150))

# KDP (computed)

axKdp = figRate.add_subplot(222)
displayRate.plot_ppi('ZDR', 0, vmin=0, vmax=2.,
    axislabels=("x(km)", "y(km)"),
    colorbar_label="ZDR (dB)",
    cmap="nipy_spectral")
displayRate.plot_range_rings([50, 100, 150, 200, 250])
displayRate.plot_cross_hair(150.)
displayRate.set_limits(xlim=(-150,150),ylim=(-150,150))

# RATE_HYBRID (computed)

axHybrid = figRate.add_subplot(223)
displayRate.plot_ppi('RHOHV', 0, vmin=0.8, vmax=1.05,
    axislabels=("x(km)", "y(km)"),
    colorbar_label="RHOHV")
displayRate.plot_range_rings([50, 100, 150, 200, 250])
displayRate.plot_cross_hair(150.)
displayRate.set_limits(xlim=(-150,150),ylim=(-150,150))

# NCAR PID (computed)

axPID = figRate.add_subplot(224)
displayRate.plot_ppi('NCP', 0, vmin=0.4, vmax = 1.1,
    axislabels=("x(km)", "y(km)"),
    colorbar_label="NCP",
    mask_outside=True)
displayRate.plot_range_rings([50, 100, 150, 200, 250])
displayRate.plot_cross_hair(150.)
displayRate.set_limits(xlim=(-150,150),ylim=(-150,150))

figRate.tight_layout()

plt.show()


# 5. Extras

<a id=’RadxRate-parameters’></a>
## **RadxRate parameters**

Below is a list of important RadxRate parameters. The line numbers are accurate as of Dec 2022 for the final Topaz release of LROSE. 

* **RadxRate main parameters**
    * input_dir [line 105]: path to input data, can be specified on command line
    * INPUT FIELD INFORMATION [line 155]: must match input file variable names
        * here SNR_available = FALSE [line 177]
        * here DBZ_field_name = "REF" [line 215]
        * here LDR_available = FALSE [line 255]
    * kdp_params_file_path [line 284]: path to Kdp-specific parameter file
    * pid_params_file_path [line 303]: path to PID-specific parameter file
    * PID_use_attenuation_corrected_fields [line 331]: specify whether to use attenuation-corrected DBZ and ZDR in PID
    * RATE_params_file_path [line 350]: path to precipitation rate coefficients parameter file
    * RATE_use_attenuation_corrected_fields [line 378]: specify whether to use attenuation-corrected DBZ and ZDR in precipitation rate calculation
    * output_dir [line 699]: path where output files are written, can set on command line with -outdir
    * output_format [line 755]: output file format, usually CfRadial
* **RadxRate Kdp_specific parameters**
    * KDP_fir_filter_len [line 65]: filter length used for KDP calculation
        * here KDP_FIR_LEN_10
    * KDP_psob_method [line 114]: specify method to remove phase shift on backscatter
        * here PEAK_REMOVAL_METHOD
* **RadxRate Pid_specific parameters**
    * PID_thresholds_file_path [line 27]: path to fuzzy logic PID thresholds file
    * PID_use_soundings_from_spdb [line 303]: specify whether soundings are in Spdb format, otherwise sounding found in fuzzy logic file
        * here TRUE
    * PID_sounding_spdb_url [line 313]: path to Spdb soundings
    * PID_sounding_location_name [line 339]: name of sounding location
        * here $(RADAR_NAME)
* **RadxRate Rate_specific parameters**
    * RATE_zh_aa [line 154]: R(Z) coefficients
    * RATE_zh_bb [line 164]: R(Z) coefficients
    * RATE_zh_aa_snow [line 182]: R(Z) coefficients in snow
    * RATE_zh_bb_snow [line 192]: R(Z) coefficients in snow
    * RATE_zzdr_aa [line 210]: R(Z, ZDR) coefficients
    * RATE_zzdr_bb [line 220]: R(Z, ZDR) coefficients
    * RATE_zzdr_cc [line 230]: R(Z, ZDR) coefficients
    * RATE_kdp_aa [line 248]: R(KDP) coefficients
    * RATE_kdp_bb [line 258]: R(KDP) coefficients
    * RATE_kdpzdr_aa [line 277]: R(KDP, ZDR) coefficients
    * RATE_kdpzdr_bb [line 287]: R(KDP, ZDR) coefficients
    * RATE_kdpzdr_cc [line 297]: R(KDP, ZDR) coefficients
* **Pid Thresholds parameters**
    * Radar dependent
        * NEXRAD is S-band simultaneous h/v transmission and recieving
        * Other parameter files can be found [here](http://wiki.lrose.net/index.php/RadxPid#PID_thresholds_file)
